In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from vkoga_2L import VKOGA_2L
import tkernels
import kernels

In [2]:
def generate_data(n_samples=1000, n_features=2, n_targets=1, noise=False):
    desired_A = torch.tensor([[0.5, 0.0], [0.0, 2.0]])  # True transformation matrix to calc X and compare later to learned A
    X = torch.randn(n_samples, n_features)
    X_transformed = X @ desired_A
    noise_data = torch.zeros(n_samples)
    if noise:
        noise_data = torch.randn(n_samples)
    y = X_transformed[:, 0] + X_transformed[:, 1] + 0.1 * noise_data
    return X, y, desired_A

In [3]:
# Generate data
X, Y, desired_A = generate_data(n_samples=1000, n_features=2, n_targets=1, noise=True)

# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [4]:
# Initialize VKOGA_2L model (some Parameters are described in Page 123)
model = VKOGA_2L(
    kernel=[kernels.Matern(k=2), tkernels.Matern(k=2)], # quadratic Matern kernel used
    flag_2L_optimization=True,
    verbose=False,
    greedy_type='f_greedy',
    reg_par=0,
    restr_par=1e-2,
    tol_f=1e-10,
    tol_p=1e-10,
    reg_para_optim=1e-3,
    learning_rate=5e-3,
    n_epochs_optim=1000,
    batch_size=32,
    flag_optim_verbose=False
)

In [5]:
# Fit the model on training data
model.fit(X_train, y_train)

VKOGA_2L(flag_2L_optimization=True, flag_optim_verbose=False,
         kernel=<kernels.Matern object at 0x00000254BAE092B0>,
         n_epochs_optim=1000, restr_par=0.01, verbose=False)

In [6]:
# Print some predictions and ground truth
predictions = model.predict(X_test)
predictions = predictions.flatten()
print("Predictions:", predictions[:5])
print("Ground truth:", y_test[:5])

# Print Error
mse = np.mean((predictions - y_test.numpy()) ** 2)
print(f"MSE on Test Data: {mse:.6f}")
print(f"Transformationmatrix: {model.A.round(decimals=3)}")
U, S, VT = np.linalg.svd(model.A.round(decimals=3))
print("Singulärwerte (S):", S.round(decimals=3))
print("Linke singuläre Vektoren (U):\n", U.round(decimals=3))
print("Rechte singuläre Vektoren (V^T):\n", VT.round(decimals=3))

mse_sample = np.mean((predictions[:5] - y_test[:5].numpy()) ** 2)
print(f"MSE on First 5 Samples: {mse_sample:.6f}")


Predictions: [-2.67114258  1.23193359  0.47949219  1.53845215 -2.10107422]
Ground truth: tensor([-2.7937,  1.4943,  0.6867,  1.2488, -2.0089])
MSE on Test Data: 0.053923
Transformationmatrix: [[-0.023 -0.006]
 [-0.051  0.039]]
Singulärwerte (S): [0.066 0.018]
Linke singuläre Vektoren (U):
 [[-0.241 -0.971]
 [-0.971  0.241]]
Rechte singuläre Vektoren (V^T):
 [[ 0.834 -0.552]
 [ 0.552  0.834]]
MSE on First 5 Samples: 0.043846


In [7]:
# No use of 2 Layer Vkoga (No learned A => A is identity)
# Initialize VKOGA_2L model (some Parameters are described in Page 123)
model = VKOGA_2L(
    kernel=[kernels.Matern(k=2), tkernels.Matern(k=2)], # quadratic Matern kernel used
    flag_2L_optimization=False,
    verbose=False,
    greedy_type='f_greedy',
    reg_par=0,
    restr_par=1e-2,
    tol_f=1e-10,
    tol_p=1e-10,
    reg_para_optim=1e-5,
    learning_rate=1e-3,
    n_epochs_optim=1000,
    batch_size=32,
    flag_optim_verbose=False
)

If kernel optimization is desired, set flag_2L_optimization to True!


In [8]:
# Fit the model on training data
model.fit(X_train, y_train)

VKOGA_2L(flag_optim_verbose=False,
         kernel=<kernels.Matern object at 0x00000254BC599D30>,
         learning_rate=0.001, n_epochs_optim=1000, reg_para_optim=1e-05,
         restr_par=0.01, verbose=False)

In [9]:
# Print some predictions and ground truth
predictions = model.predict(X_test)
predictions = predictions.flatten()
print("Predictions:", predictions[:5])
print("Ground truth:", y_test[:5])

# Print Error
mse = np.mean((predictions - y_test.numpy()) ** 2)
print(f"MSE on Test Data: {mse:.6f}")
print(f"Transformationmatrix: {model.A.round(decimals=3)}")

Predictions: [-2.71238859  1.45945191  0.55231721  1.28563785 -2.04946843]
Ground truth: tensor([-2.7937,  1.4943,  0.6867,  1.2488, -2.0089])
MSE on Test Data: 0.056268
Transformationmatrix: [[1. 0.]
 [0. 1.]]
